<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/QAS_BERT_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import os 
import sys 
import json
from platform import python_version

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,Dense,MultiHeadAttention

import tensorflow_hub as hub
!pip install -q tensorflow-text

import tensorflow_text as text

In [2]:
print('Python Version : ' ,python_version())
print('tensorflow version : ', tf.__version__)
print('pandas version : ', pd.__version__)
print('numpy version : ', np.__version__)
print('tf_hub version : ', hub.__version__)


Python Version :  3.7.10
tensorflow version :  2.4.1
pandas version :  1.1.5
numpy version :  1.19.5
tf_hub version :  0.11.0


In [3]:
train_path = keras.utils.get_file("train.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")
eval_path = keras.utils.get_file("eval.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json")
with open(train_path) as f: raw_train_data = json.load(f)
with open(eval_path) as f: raw_eval_data = json.load(f)
# max_seq_length = 384

In [4]:
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['title'][0:])
print(raw_train_data['data'][1]['title'][0:])

dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
University_of_Notre_Dame
Beyoncé


In [5]:
print(raw_train_data['data'][3]['paragraphs'][0].keys())

dict_keys(['context', 'qas'])


In [6]:
print(raw_train_data['data'][0]['paragraphs'][0]['qas'])

[{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}, {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ'}], 'question': 'What is in front of the Notre Dame Main Building?', 'id': '5733be284776f4190066117f'}, {'answers': [{'answer_start': 279, 'text': 'the Main Building'}], 'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?', 'id': '5733be284776f41900661180'}, {'answers': [{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}], 'question': 'What is the Grotto at Notre Dame?', 'id': '5733be284776f41900661181'}, {'answers': [{'answer_start': 92, 'text': 'a golden statue of the Virgin Mary'}], 'question': 'What sits on top of the Main Building at Notre Dame?', 'id': '5733be284776f4190066117e'}]


In [7]:
print(raw_train_data['data'][0]['title'])
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['paragraphs'][0]['qas'][1]['answers'][0]['text'])
x = (raw_train_data['data'][0]['paragraphs'][0]['context'])
x

University_of_Notre_Dame
dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
a copper statue of Christ


'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [8]:
def text_maker (data):
  squad_examples = []
  full_context = []
  full_question = []
  full_answer = []
  title = []
  starting_index = []
  for item in data["data"]:
    # full_context = []
      title.append(item['title'])
      for para in item["paragraphs"]:
          context = para["context"]
          full_context.append(context)
          for qa in para["qas"]:
              question = qa["question"]
              full_question.append(question)
              if "answers" in qa:
                  answer_text = qa["answers"][0]["text"]
                  full_answer.append(answer_text)
                  all_answers = [_["text"] for _ in qa["answers"]]
                  start_char_idx = qa["answers"][0]["answer_start"]
                  # print(start_char_idx)
                  starting_index.append(start_char_idx)
                # squad_eg = Sample(question, context, start_char_idx, answer_text, all_answers)
            # else:
                # squad_eg = Sample(question, context)
            # squad_eg.preprocess()
            # squad_examples.append(squad_eg)

  return(full_context, full_question, full_answer,starting_index,title)

In [9]:
train_context, train_question,train_answer,train_answer_index,train_titles= text_maker(raw_train_data)
test_context, test_question, test_answer,test_answer_index,test_titles = text_maker(raw_eval_data)

In [10]:
print('length of context', len(train_context))
print('length of the question', len(train_question))
print('length of the answers', len(train_answer))
print('length of the index', len (train_answer_index))
print('length of the title', len(train_titles))

length of context 18896
length of the question 87599
length of the answers 87599
length of the index 87599
length of the title 442


In [11]:
bert_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_model = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

In [12]:
bert_preprocess_layer = hub.load(bert_preprocess)
bert_layer = hub.KerasLayer(bert_model,trainable=True)

In [13]:
# text_test = ['this is such an amazing movie!']

# sample_tokenizer = bert_preprocess_layer(text_test)
# print(f'Keys       : {list(sample_tokenizer.keys())}')
# print(f'Shape      : {sample_tokenizer["input_word_ids"].shape}')
# print(f'Word Ids   : {sample_tokenizer["input_word_ids"][0, :12]}')
# print(f'Input Mask : {sample_tokenizer["input_mask"][0, :12]}')
# print(f'Type Ids   : {sample_tokenizer["input_type_ids"][0, :12]}')

In [14]:
train_context[0]
train_question[0]


'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [22]:
x = bert_preprocess_layer.tokenize(tf.constant(['I live in london']))

In [16]:
bert_preprocess_layer.tokenize(['I live in london', 'I live in london'])


<tf.RaggedTensor [[[1045], [2444], [1999], [2414]], [[1045], [2444], [1999], [2414]]]>

In [17]:
a = tf.constant(train_context[0])
b = tf.constant(train_question[0])

In [24]:
bert_preprocess_layer.bert_pack_inputs([x,x])

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 